In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [6]:
#Load movie_metadata CSV file
start_file = pd.read_csv("Resource/movie_metadata.csv")

#Remove extraneous columns
start_file = start_file.drop(columns = ["color", "num_critic_for_reviews", "director_facebook_likes", 
                                       "actor_3_facebook_likes", "actor_1_facebook_likes", "cast_total_facebook_likes",
                                       "plot_keywords", "actor_2_facebook_likes", "aspect_ratio", 
                                       "num_voted_users", "num_user_for_reviews", "movie_imdb_link"])

Index(['director_name', 'duration', 'actor_2_name', 'gross', 'genres',
       'actor_1_name', 'movie_title', 'actor_3_name', 'facenumber_in_poster',
       'language', 'country', 'content_rating', 'budget', 'title_year',
       'imdb_score', 'movie_facebook_likes'],
      dtype='object')

In [15]:
#Remove extra spaces at end of movie titles
start_file["movie_title"] = start_file["movie_title"].str.rstrip()

'Raymond Did It'

In [30]:
#Find films where country is missing; identify the ones made in USA
start_file.loc[start_file["country"].isnull()]
start_file.at[4, "country"] = "USA"
start_file.at[279, "country"] = "USA"
start_file.at[3397, "country"] = "USA"
start_file.at[4021, "country"] = "USA"

,director_name,duration,actor_2_name,gross,genres,actor_1_name,movie_title,actor_3_name,facenumber_in_poster,language,country,content_rating,budget,title_year,imdb_score,movie_facebook_likes
2370,NaN,43.0,Jenni 'Jwoww' Farley,NaN,Comedy|Drama|Reality-TV|Romance,Nicole 'Snooki' Polizzi,"Gone, Baby, Gone",Paul 'Pauly D' DelVecchio,1.0,English,NaN,TV-14,NaN,NaN,6.6,0


In [37]:
#Find films made in the USA
start_file.loc[start_file["country"] == "USA"]

,director_name,duration,actor_2_name,gross,genres,actor_1_name,movie_title,actor_3_name,facenumber_in_poster,language,country,content_rating,budget,title_year,imdb_score,movie_facebook_likes
0,James Cameron,178.0,Joel David Moore,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,Wes Studi,0.0,English,USA,PG-13,237000000.0,2009.0,7.9,33000
1,Gore Verbinski,169.0,Orlando Bloom,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,Jack Davenport,0.0,English,USA,PG-13,300000000.0,2007.0,7.1,0
3,Christopher Nolan,164.0,Christian Bale,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,Joseph Gordon-Levitt,0.0,English,USA,PG-13,250000000.0,2012.0,8.5,164000
4,Doug Walker,NaN,Rob Walker,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens,NaN,0.0,NaN,USA,NaN,NaN,NaN,7.1,0
5,Andrew Stanton,132.0,Samantha Morton,73058679.0,Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,Polly Walker,1.0,English,USA,PG-13,263700000.0,2012.0,6.6,24000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5037,Edward Burns,95.0,Caitlin FitzGerald,4584.0,Comedy|Drama,Kerry Bishé,Newlyweds,Daniella Pineda,1.0,English,USA,Not Rated,9000.0,2011.0,6.4,413
5039,NaN,43.0,Valorie Curry,NaN,Crime|Drama|Mystery|Thriller,Natalie Zea,The Following,Sam Underwood,1.0,English,USA,TV-14,NaN,NaN,7.5,32000
5040,Benjamin Roberds,76.0,Maxwell Moody,NaN,Drama|Horror|Thriller,Eva Boehnke,A Plague So Pleasant,David Chandler,0.0,English,USA,NaN,1400.0,2013.0,6.3,16
5041,Daniel Hsia,100.0,Daniel Henney,10443.0,Comedy|Drama|Romance,Alan Ruck,Shanghai Calling,Eliza Coupe,5.0,English,USA,PG-13,NaN,2012.0,6.3,660
